In [15]:
import pandas as pd
from sklearn import svm
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense

In [3]:
data = pd.read_csv("german_encoded.csv")

In [4]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# SVM

In [56]:
def linear():
    model = LinearSVC()
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
    print('CV score:', scores.mean(), "+/-", scores.std())
    
linear()

CV score: 0.6781854651312967 +/- 0.04357108294872709


Results:
- CV score: 0.6802346253798887 +/- 0.0416567058131753
- CV score: 0.6781117500052598 +/- 0.04217883266936663
- CV score: 0.6781854651312967 +/- 0.04357108294872709

In [57]:
model = SVC()
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
print('CV score:', scores.mean(), "+/-", scores.std())

CV score: 0.6784608290896231 +/- 0.058583521857931885


Results:
- CV score: 0.6784608290896231 +/- 0.058583521857931885

## GridSearch

In [67]:
param_grid= {'kernel': ('linear', 'rbf'),'C': [10, 20, 30, 40, 100], 'gamma': [0.05, 0.01, 0.005, 0.001]}
base_estimator = SVC()
sh = GridSearchCV(base_estimator, param_grid, cv=10,scoring = 'f1_macro').fit(X_train, y_train)

In [68]:
param = sh.best_params_
score = sh.score(X_train, y_train)
print("Best param:", param)
print("Best score:",score)

Best param: {'C': 30, 'gamma': 0.005, 'kernel': 'rbf'}
Best score: 0.7821060015919842


Results:
- {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}, score = 0.7983922807033947
- {'C': 20, 'gamma': 0.01, 'kernel': 'rbf'}, score = 0.8290713574757445
- {'C': 30, 'gamma': 0.005, 'kernel': 'rbf'}, score = 0.7821060015919842

# SGD

In [60]:
model = SGDClassifier(loss = 'log_loss')

scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
print(scores.mean())

0.6165564027326179


Results:
- 0.5615924038004938
- 0.6265978496656802
- 0.6165564027326179

In [69]:
param_grid = {
    'alpha': [0.0001, 0.00005, 0.00007, 0.00009],
    'penalty': ['l2','l1'],
    'eta0': [0.0001, 0.001, 0.005, 0.007, 0.009, 0.01, 0.015]
}

# create a GridSearchCV object with the specified hyperparameter grid
grids = GridSearchCV(model, param_grid, cv=10, scoring = 'f1_macro', verbose = 1)

# fit the grid search object to the data
grids.fit(X_train , y_train)

# print the best hyperparameters and the corresponding score
print("Best hyperparameters: ", grids.best_params_)
print("Best score: ", grids.best_score_)

Fitting 10 folds for each of 56 candidates, totalling 560 fits


ValueError: Invalid parameter 'alpha' for estimator RandomForestClassifier(). Valid parameters are: ['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].

Results:
- {'alpha': 7e-05, 'eta0': 0.005, 'penalty': 'l2'}, score = 0.6616935571076444
- {'alpha': 7e-05, 'eta0': 0.001, 'penalty': 'l1'}, score = 0.6270937017313281

# RandomForest

In [18]:
model = RandomForestClassifier()
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1_macro', error_score="raise")
print('CV score:', scores.mean(), "+/-", scores.std())

CV score: 0.6332005407999604 +/- 0.054837880641444355


Results:
- CV score: 0.663357274635801 +/- 0.04059552799388036
- CV score: 0.6464275844463873 +/- 0.04375839610392233
- CV score: 0.6578586265934101 +/- 0.04422778600545267

In [20]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [25]:
from sklearn.tree import export_text
# Print out the trees in the ensemble
for i, tree in enumerate(model.estimators_):
    print(f"Tree {i}:\n{export_text(tree)}\n")

Tree 0:
|--- feature_1 <= 2.50
|   |--- feature_17 <= 1.50
|   |   |--- feature_19 <= 0.50
|   |   |   |--- feature_0 <= 2.50
|   |   |   |   |--- feature_12 <= 1.50
|   |   |   |   |   |--- feature_5 <= 1.50
|   |   |   |   |   |   |--- feature_6 <= 1.50
|   |   |   |   |   |   |   |--- feature_5 <= 0.50
|   |   |   |   |   |   |   |   |--- feature_7 <= 1.50
|   |   |   |   |   |   |   |   |   |--- feature_2 <= 1.50
|   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |--- feature_2 >  1.50
|   |   |   |   |   |   |   |   |   |   |--- feature_8 <= 2.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |   |   |   |   |   |--- feature_8 >  2.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |--- feature_7 >  1.50
|   |   |   |   |   |   |   |   |   |--- feature_16 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |--- featur

In [27]:
# Get the list of decision tree estimators
estimators = model.estimators_

# Find the best decision tree
best_tree = None
best_score = -1

for estimator in estimators:
    score = estimator.score(X_train, y_train)
    if score > best_score:
        best_tree = estimator
        best_score = score

# Print the best tree
tree_str = export_text(best_tree)
print(tree_str)

|--- feature_0 <= 1.50
|   |--- feature_6 <= 1.50
|   |   |--- feature_5 <= 3.50
|   |   |   |--- feature_19 <= 0.50
|   |   |   |   |--- feature_8 <= 0.50
|   |   |   |   |   |--- class: 1.0
|   |   |   |   |--- feature_8 >  0.50
|   |   |   |   |   |--- feature_2 <= 3.50
|   |   |   |   |   |   |--- feature_14 <= 1.50
|   |   |   |   |   |   |   |--- feature_10 <= 3.50
|   |   |   |   |   |   |   |   |--- feature_12 <= 0.50
|   |   |   |   |   |   |   |   |   |--- feature_2 <= 1.50
|   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |--- feature_2 >  1.50
|   |   |   |   |   |   |   |   |   |   |--- feature_7 <= 2.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |   |--- feature_7 >  2.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 4
|   |   |   |   |   |   |   |   |--- feature_12 >  0.50
|   |   |   |   |   |   |   |   |   |--- feature_11 <= 2.50
|   |   |   |   |

/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/opt/pub/eb/apps/all/DavidsonJupyter/2021-common-GCCcore-10.2.0/lib/python3.8/site-packages/sklearn/base

# k-NN

In [70]:
param_grid= {'n_neighbors': [15,20,25],'p': [1,2]}
models = KNeighborsClassifier()
ch = GridSearchCV(models, param_grid, cv=10,scoring = 'f1_micro').fit(X_train, y_train)

In [71]:
k_param = ch.best_params_
k_score = ch.best_score_
print('best param is', k_param)
print('best score is', k_score)

best param is {'n_neighbors': 20, 'p': 1}
best score is 0.752


Results:
- {'n_neighbors': 11, 'p': 1}, score = 0.7506666666666667
- {'n_neighbors': 20, 'p': 1}, score = 0.752

# Decision Tree

In [65]:
# Define the decision tree classifier and the hyperparameters to optimize
clf = DecisionTreeClassifier()
param_grid = {'max_depth': [7, 8, 9],
              'min_samples_split': [9, 10, 11],
              'min_samples_leaf': [1, 2, 3, 4, 5]}

# Perform the grid search with cross-validation
dtgs = GridSearchCV(clf, param_grid=param_grid, cv=10,scoring = 'f1_micro')
dtgs.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding evaluation metric
print("Best hyperparameters:", dtgs.best_params_)
print("Best score:", dtgs.best_score_)

Best hyperparameters: {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10}
Best score: 0.72


Results:
- {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10}, score = 0.7186666666666667
- {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10}, score = 0.7173333333333333
- {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10}, score = 0.72

# Neural Network

In [66]:
# build the neural network model
nn = Sequential()
nn.add(Dense(10, input_dim=20, activation='relu'))
nn.add(Dense(5, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))

# compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model to the training data
nn.fit(X_train, y_train, epochs=50, batch_size=32)

# evaluate the model on the test data
loss, accuracy = nn.evaluate(X_train, y_train)

# print the test accuracy
print('Test accuracy:', accuracy)

Epoch 1/50
24/24 [==============================] - 1s 1ms/step - loss: -0.2201 - accuracy: 0.6973
Epoch 2/50
24/24 [==============================] - 0s 1ms/step - loss: -0.6296 - accuracy: 0.7000
Epoch 3/50
24/24 [==============================] - 0s 1ms/step - loss: -0.9763 - accuracy: 0.7000
Epoch 4/50
24/24 [==============================] - 0s 1ms/step - loss: -1.4045 - accuracy: 0.7000
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: -2.1002 - accuracy: 0.7000
Epoch 6/50
24/24 [==============================] - 0s 1ms/step - loss: -3.2095 - accuracy: 0.7000
Epoch 7/50
24/24 [==============================] - 0s 1ms/step - loss: -4.8941 - accuracy: 0.7000
Epoch 8/50
24/24 [==============================] - 0s 1ms/step - loss: -7.1664 - accuracy: 0.7000
Epoch 9/50
24/24 [==============================] - 0s 1ms/step - loss: -10.1646 - accuracy: 0.7000
Epoch 10/50
24/24 [==============================] - 0s 1ms/step - loss: -14.1715 - accuracy: 0.7000
Epoch 1

Results:
- Test accuracy: 0.699999988079071, 50 epochs, 10 in second layer, batch size = 32
- Test accuracy: 0.699999988079071, 70 epochs, 10 in second layer, batch size = 40
- Test accuracy: 0.699999988079071, 50 epochs, 10 in second layer, 5 in third layer, batch size = 32

# Scores

SVM: {'C': 20, 'gamma': 0.01, 'kernel': 'rbf'}, score = 0.8290713574757445

SGD: {'alpha': 7e-05, 'eta0': 0.005, 'penalty': 'l2'}, score = 0.6616935571076444

RandomForest: CV score: 0.663357274635801 +/- 0.04059552799388036

k-NN: {'n_neighbors': 20, 'p': 1}, score = 0.752

Decision Tree: {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 10}, score = 0.72

Neural Network: Test accuracy: 0.699999988079071, 50 epochs, 10 in second layer, batch size = 32

**SVM is the highest**